In [42]:
import numpy as np
from scipy.io import loadmat
from statsmodels import robust
import mne
from scipy import signal

def performPrep(eeg, eog, refChan, srate, linenoise, referenceType='robust'):

    if refChan != 0:
        dim = np.shape(eeg)
        eeg_chans=np.setdiff1d(range(0, dim[0]), refChan-1) #remove the reference channel from the eeg channels
        eeg=eeg[eeg_chans,:]

    eeg=np.row_stack((eeg,eog)) #combine the eeg and eog matrices row-wise
    #perform high pass filtering and detrending

    mne.filter.filter_data(eeg, srate, 1, None, picks=None, filter_length='auto', l_trans_bandwidth='auto',
                           h_trans_bandwidth='auto', n_jobs=1, method='fir', iir_params=None, copy=True, phase='zero',
                           fir_window='hamming', fir_design='firwin', pad='reflect_limited', verbose=None)
    # performFilter(eeg,srate,filter_type='high',filter_freq=1)
    eeg = signal.detrend(eeg)

    #removing line noise
    mne.filter.notch_filter(eeg, srate, linenoise, filter_length='auto', notch_widths=None, trans_bandwidth=1, method='fir',
                            iir_params=None, mt_bandwidth=None, p_value=0.05, picks=None, n_jobs=1, copy=True,
                            phase='zero', fir_window='hamming', fir_design='firwin', pad='reflect_limited',
                            verbose=None)
    # performFilter(eeg,srate,filter_type='notch',filter_freq=50)

    #finding bad channels

    #finding channels with NaNs or constant values for long periods of time
    dim = np.shape(eeg)
    channelsInterpolate = np.array(range(1, dim[0] + 1))
    nanChannelMask = np.zeros(dim[0])
    noSignalChannelMask = np.zeros(dim[0])
    badChannelsfromNans = np.zeros(dim[0])
    badChannelsfromNoData = np.zeros(dim[0])
    robustchanneldeviation = np.zeros(dim[0])
    badChannelFromDeviationMask = np.zeros(dim[0])
    badChannelFromDeviation=np.zeros(dim[0])
    channeldeviation = np.zeros(dim[0])
    for i in range(0, dim[0]):
        nanChannelMask[i] = np.int(np.sum(np.isnan(eeg[i, :])) > 0)
    for i in range(0, dim[0]):
        noSignalChannelMask[i] = np.int(robust.mad(eeg[i, :]) < 10 ** (-10) or np.std(eeg[i, :]) < 10 ** (-10))
    print(noSignalChannelMask)
    for i in range(0, dim[0]):
        if nanChannelMask[i] == 1:
            badChannelsfromNans[i] = i + 1
        if noSignalChannelMask[i] == 1:
            badChannelsfromNoData[i] = i + 1
    channelsInterpolate = np.setdiff1d(channelsInterpolate, np.union1d(badChannelsfromNans, badChannelsfromNoData)) #channels to be used for interpolation

    # find channels that have abnormally high or low amplitude
    for i in range(0, dim[0]):
        channeldeviation[i] = 0.7413 * (np.percentile(eeg[i, :], 75) - np.percentile(eeg[i, :], 25))

    channeldeviationSD = 0.7413 * (np.percentile(channeldeviation, 75) - np.percentile(channeldeviation, 25))
    channeldeviationMedian = np.nanmedian(channeldeviation)
    for i in range(0, dim[0]):
        robustchanneldeviation[i] = (channeldeviation[i] - channeldeviationMedian) / channeldeviationSD

    for i in range(0, dim[0]):
        badChannelFromDeviationMask[i] = np.int(abs(robustchanneldeviation[i]) > 5 or np.isnan(robustchanneldeviation[i]))
        if badChannelFromDeviationMask[i] == 1:
            badChannelFromDeviation[i]=i+1
    print(badChannelsfromNoData)
    print(badChannelFromDeviation)
    print(badChannelsfromNans)
    
    badChannels=np.union1d(np.union1d(badChannelsfromNoData, badChannelFromDeviation), badChannelsfromNans)

    return badChannels

In [43]:
import numpy as np
import mne
import pytest 
from ast import literal_eval
import matplotlib.pyplot as plt

chanType = ['eeg'] * 22 + ['eog'] * 3
x = list(range(1,26))
channNames = [str(i) for i in x]
info = mne.create_info(channNames,250,chanType)
data = mne.io.read_raw_edf('/Users/raphaelbechtold/Documents/MATLAB/Automagic/automagic/data/Subj1/S001R01.edf')
eeg, time = data[:]
#eeg[5,::] = -1*np.ones(eeg.shape[1])
badchannels = performPrep(eeg, np.zeros(eeg.shape[1]), 0, 250, 60)
print('badChannels')
print(badchannels)


def test_sample_input():
    data = mne.io.read_raw_edf('/Users/raphaelbechtold/Documents/MATLAB/Automagic/automagic/data/Subj1/S001R01.edf')
    eeg, time = data[:]
    print(time)
    eeg[5,::] = -1*np.ones(eeg.shape[1])
    badchannels = performPrep(eeg, NaN, 0, 250, 60)

Extracting EDF parameters from /Users/raphaelbechtold/Documents/MATLAB/Automagic/automagic/data/Subj1/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 825 samples (3.300 sec)

Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband e